# Types

1. **Static type systems**
    - Every program expression must have a type computable before the execution of the program.
1. **Dynamic type systems**
    - Nothing is known about types until run time when the actual values manipulated by the program are available.
    
    >*So, what roles type play at the compile time in Julia?*
    
**Polymorphism**: the ability to write code that can operate on different types.

## Julia's Type system: _**Dynamic**_, _**Nominative**_ and _**Parametric**_

- Some references:
    - [Nominative And Structural Typing](http://wiki.c2.com/?NominativeAndStructuralTyping)
    - [Type system](https://en.wikipedia.org/wiki/Type_system)

**Julia's type system is dynamic, but it possible to indicate that certain values are of specific types.**

- Default behavior when types are omitted: allow values to be of any type.
- Adding annotations serves three primary purposes:
    1. to take advantage of Julia's multiple-dispatch mechanism
    1. to improve human readability
    1. to catch programmer errors

Benefits of Julia's type system design:

1. Great assistance in generating efficient code.
1. [_**more significantly**_ ??] Allows method dispatch on the types of function arguments to be deeply integrated with the language.
    - Method dispatch is rooted in the type system presented here.
    
### Notes (and how to understand these concretely?)

1. All values in Julia are **true objects** having a type that belongs to a single, fully connected type graph, all nodes of which are equally first-class as types.

   > In mainstream object oriented languages, such as C++, Java, Python and Ruby, [composite types](https://en.wikipedia.org/wiki/Composite_data_type) also have named functions associated with them, and the combination is called an "object".

1. There is no meaningful concept of a "compile-time type": the **only type** a value has is its actual _**type when the program is running**_.

    - _**So how to understand "type" in static analysis (before codes are compiled), and what role it plays before codes are compiled? Maybe have something to do with multiple dispatch?**_

1. Only values, not variables, have types – variables are simply names bound to values.
1. Both abstract and concrete types can be parameterized by symbols, by values of any type for which `isbits` returns true.

## Type Declarations

The `::` operator can be used to attach type annotations to _**expressions**_ and variables in programs. 

- When appended to an expression computing a value, the `::` operator is read as "is an instance of". 

## Special Types

### 1. _Abstract type: Type Union_

- A type union is a special abstract type.
- Type union is constructed using the special `Union` keyword.

    ```julia
    IntOrString = Union{Int, AbstractString}
    ```
- Type union has parameters. It is a type includes all instances of any of its argument types.

_**Why Union type?**_

- The compilers for many languages have an internal union construct for reasoning about types; Julia simply exposes it to the programmer.
- The Julia compiler is able to generate efficient code in the presence of Union types with a small number of types.

#### Related keywords or built-in types

- 1.1 [Core.Union](https://docs.julialang.org/en/v1/base/base/#Core.Union) _- Type_

    A type union is an _**abstract type**_ which includes all instances of any of its argument types. The empty union `Union{}` is _**the bottom type of Julia**_.

- 1.2 [where](https://docs.julialang.org/en/v1/base/base/#where)

    >Their issue: https://github.com/JuliaLang/julia/issues/20337

    The `where` keyword creates a type that is an iterated union of other types, over all values of some variable. 

In [7]:
Vector{T} where T <: Real

Array{T,1} where T<:Real

### 2. Tuple

- Tuples are an abstraction of _**the arguments of a function**_ – without the function itself.
- The salient aspects of a function's arguments are _**their order and their types**_.
    - A tuple type is similar to _**a parameterized immutable type**_ where each parameter is the type of one field.
    - Tuple types may have any number of parameters.

**Tuple types are [covariant](https://en.wikipedia.org/wiki/Covariance_and_contravariance_(computer_science)) in their parameters**:
- `Tuple{Int}` is a subtype of `Tuple{Any}`. `Tuple{Any}` is considered an abstract type.
- Tuple types are only concrete if their parameters are.
- Tuples **do not** have field names; fields are **only accessed** by index.


In composite type, This last point is very important: even though Float64 <: Real we DO NOT have Point{Float64} <: Point{Real}.

In [15]:
@show Float64 <: Real

struct Point{T}
  x::T
end
@show Point{Float64} <: Point{Real}

a = Tuple{Real}
b = Tuple{Float64}
@show b <: a;

Float64 <: Real = true
Point{Float64} <: Point{Real} = false
b <: a = true


### 3. [Singleton Type](https://docs.julialang.org/en/v1/manual/types/#man-singleton-types-1)

- For each type, `T`, the "singleton type" `Type{T}` is an **abstract type** whose only instance is the object `T`.
- Without the parameter, `Type` is simply an abstract type which has **all type objects** as its instances, including, of course, singleton types.

In [22]:
@show Float32 <: Type
@show Type <: Type    # except Type, no other type subtype the singleton `Type`. 
@show isa(Float32, Type);

Float32 <: Type = false
Type <: Type = true
Float32 isa Type = true


## Commposite Types

Composite objects declared with struct are _**immutable**_; they cannot be modified after construction.

In [43]:
struct TestType1
    a::Int32
    b::Float32
end

test = TestType1(5, 5.5)
@show typeof(test)
@show test.a
@show test.b;

typeof(test) = TestType1
test.a = 5
test.b = 5.5f0


In [44]:
# after test is constructed, it cannot be modified.
# this will cause an error.
test.a = 6

ErrorException: type TestType1 is immutable

In [2]:
struct TestType2
  a::Array{Int32, 1}  # Array is a mutable type.
end

test2 = TestType2([1, 2])

TestType2(Int32[1, 2])

In [4]:
# for fields whose values are mutable types like arrays
# the fields will always refer to the same mutable value even though that mutable value's content may itself be modified.
test2.a = [3, 4]

ErrorException: type TestType2 is immutable

**Mutable values are heap-allocated and passed to functions as pointers to heap-allocated values except in cases where the compiler is sure that there's no way to tell that this is not what is happening**.

## Types Internals

1. _**Types are regular objects at runtime**_, that are internally represented as instances of the same concept called `DataType`.
1. `DataType` is the type of any of abstract, primitive and composite type.
    - A `DataType` may be abstract or concrete.
        - If it is concrete, it has a specified size, storage layout, and (optionally) field names.
        - A primitive type is a `DataType` with nonzero size, but no field names.
        - A composite type is a `DataType` that has field names or is empty (zero size).
1. Every concrete value in the system is an instance of some `DataType`.

In [35]:
@show typeof(Int)
@show typeof(Real)

struct A
    a
end
@show typeof(A);

typeof(Int) = DataType
typeof(Real) = DataType
typeof(A) = DataType


## _**Dynamic**_

In a dynamic language:
1. **types exist at runtime**
1. **type information can be manipulated and introspected at runtime** with the same syntax as everything else is. 

In [17]:
mytypes = [Int, Float32, Bool]  # Types are regular objects at runtime! We can put them into a list.

3-element Array{DataType,1}:
 Int64  
 Float32
 Bool   

In [21]:
a = 15

mutable struct MyType
    t1 :: supertype(Int)
    t2 :: (a > 5 ? typeof(a) : Float64)
    t3 :: mytypes[2]
end

dump(MyType)

MyType <: Any
  t1::Signed
  t2::Int64
  t3::Float32


### Notes about Julia's dynamics

_**Functions don’t really exist in dynamic languages at the point where they are run**:_
- It is at runtime when the program encounters a function definition that the code for it is actually created. In Julia’s case, it is really just **a sort of code template** which is created.
- At the time when **calling the function with arguments of specific types**, Julia will:
    1. **specialize the function**.
    1. have the JIT emit machine code specifically tailored to those particular function arguments.


- There is **no clear distinction between compile time and runtime**.
- **Functions are compiled as they are encountered at runtime**.
- _**Statically typed languages care about the type of expressions**_ while _**dynamically typed languages care about the types of values**_. 

## Nominative

- When declaring a new type, the type name can be optionally followed by `<:` and an already-existing type, indicating that the newly declared abstract type is a subtype of this "parent" type.
- By default, a new decleared type is the subtype of `Any`.

## Parameteric

References about [Parametric Polymorphism](https://en.wikipedia.org/wiki/Polymorphism_(computer_science))

>_**Parametric polymorphism**_ allows a function or a data type to be written generically, so that it can handle values uniformly without depending on their type.
>_**Parametric polymorphism is a way to make a language more expressive while still maintaining full static type-safety**_.

- Types can take parameters, so that type declarations actually _**introduce a whole family**_ of new types: one for each possible combination of parameter values. 

## Reference

1. [Types in C/C++ and Julia](https://medium.com/@Jernfrost/types-in-c-c-and-julia-ce0fcbe0dec6)